In [1]:
import random
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
import os
from fancyimpute import IterativeImputer
os.environ['PYTHONHASHSEED']=str(271828)
random.seed(271828)
np.random.seed(271828)
#import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import KernelPCA
from sklearn.pipeline import Pipeline
from tensorflow.python.framework import ops
import tensorflow as tf
tf.random.set_seed(271828)
ops.reset_default_graph()

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
#from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import keras.backend as K
#session_conf=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1) 
#sesCriticalSection=tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)

C:\Anaconda\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.
C:\Anaconda\lib\site-packages\tensorflow-2.0.0a0-py3.6-win-amd64.egg\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\lib\site-packages\tensorflow-2.0.0a0-py3.6-win-amd64.egg\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8

In [2]:

def PCL_calculator(df):

    symptomatic_cutoff = 2

    intrusion = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'q6.4_UPSET', 'q6.5_PHYS']
    avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES', 'q6.9_DISINT', 'q6.10_DTACH',
                 'q6.11_NUMB', 'q6.12_FUTRE']
    tred = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH']
    only_avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES']

    hypertension = ['q6.13_SLEEP', 'q6.14_ANGER', 'q6.15_CONC', 'q6.16_HYPER', 'q6.17_STRTL']

    depression = ['q6.9_DISINT', 'q6.10_DTACH', 'q6.11_NUMB', 'q6.12_FUTRE']

    df[intrusion + avoidance + hypertension].fillna(df[intrusion + avoidance + hypertension].mean(axis=1))
    intrusion_cuoff = 1
    avoidance_cuoff = 3
    hypertension_cuoff = 2
    only_avoidance_cutoff = 1
    depression_cutoff = 2
    tred_cutoff = 1

    df['PCL_score'] = (df[intrusion + avoidance + hypertension]).sum(axis=1)
    df['PCL_mean'] = (df[intrusion + avoidance + hypertension]).mean(axis=1)
    df['PCL_std'] = (df[intrusion + avoidance + hypertension]).std(axis=1)
    

    df['intrusion'] = (df[intrusion] > symptomatic_cutoff).sum(axis=1)
    df['intrusion_mean'] = (df[intrusion] > symptomatic_cutoff).mean(axis=1)
    df['intrusion_std'] = (df[intrusion] > symptomatic_cutoff).std(axis=1)
    df['intrusion_cutoff'] = (df['intrusion'] >= intrusion_cuoff).astype(int)

    df['avoidance'] = (df[avoidance] > symptomatic_cutoff).sum(axis=1)
    df['avoidance_mean'] = (df[avoidance] > symptomatic_cutoff).mean(axis=1)
    df['avoidance_std'] = (df[avoidance] > symptomatic_cutoff).std(axis=1)
    df['avoidance_cutoff'] = (df['avoidance'] >= avoidance_cuoff).astype(int)

    df['depression'] = (df[depression] > symptomatic_cutoff).sum(axis=1)
    df['depression_mean'] = (df[depression] > symptomatic_cutoff).mean(axis=1)
    df['depression_std'] = (df[depression] > symptomatic_cutoff).std(axis=1)
    df['depression_cutoff'] = (df['depression'] >= depression_cutoff).astype(int)

    df['hypertention'] = (df[hypertension] > symptomatic_cutoff).sum(axis=1)
    df['hypertention_mean'] = (df[hypertension] > symptomatic_cutoff).mean(axis=1)
    df['hypertention_std'] = (df[hypertension] > symptomatic_cutoff).std(axis=1)
    df['hypertention_cutoff'] = (df['hypertention'] >= hypertension_cuoff).astype(int)

    df['tred'] = (df[tred] > symptomatic_cutoff).sum(axis=1)
    df['tred_mean'] = (df[tred] > symptomatic_cutoff).mean(axis=1)
    df['tred_std'] = (df[tred] > symptomatic_cutoff).std(axis=1)
    df['tred_cutoff'] = df['tred'] >= tred_cutoff

    df['only_avoidance'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_mean'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_std'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_cutoff'] = df['only_avoidance'] >= only_avoidance_cutoff

    #df['regression_cutoff_33'] = df['sum'] >= 33
    #df['regression_cutoff_50'] = df['sum'] >= 49
    df['diagnosis'] = ((df['hypertention_cutoff']) & (df['avoidance_cutoff']) & (df['intrusion_cutoff']) & (df['PCL_score'] >= 49))

    return df



In [3]:
path = "C:\‏‏PycharmProjects\PTSD\Data\PTSD.xlsx"
df = pd.read_excel(path)
#combine with specifics of answers
df_pcl2 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL2.xlsx")
df_pcl2 = PCL_calculator(df_pcl2)

df_pcl1 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL1.xlsx")
df_pcl1 = PCL_calculator(df_pcl1)

df = df.merge(df_pcl1, on="ID", how='outer')
df = df.merge(df_pcl2, suffixes=('_pcl1', '_pcl2'), on="ID", how='outer')

df_pcl3 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL3.xlsx")
df_pcl3 = PCL_calculator(df_pcl3)
df = df.merge(df_pcl3.drop(['PCL3_Strict', 'pcl3', 'PCL3_Broad'], axis=1), on="ID", how='outer')
# rmoving missing Y's
df = df[~df['PCL_Strict3'].isna()]

In [4]:
df.columns

Index(['ID', 'age', 'Brigade', 'mother_tongue', 'highschool_diploma', 'Hebrew',
       'dyslexia', 'ADHD', 'DotProbe', 'Training',
       ...
       'hypertention_cutoff', 'tred', 'tred_mean', 'tred_std', 'tred_cutoff',
       'only_avoidance', 'only_avoidance_mean', 'only_avoidance_std',
       'only_avoidance_cutoff', 'diagnosis'],
      dtype='object', length=320)

## features 

In [5]:

features = ["highschool_diploma", "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1",
                "trait1",
                "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2",
                "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1",
                "humor1",
                "religion1", "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                "instrumental_support2",
                "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2",
                "self_blame2",
                "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", 'q6.1_INTRU_pcl1', 
                'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                  'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                  'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                  'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                  'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                  'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                  'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                  'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2',
            
            'PCL_score_pcl1', 'PCL_mean_pcl1', 'PCL_std_pcl1', 'intrusion_pcl1', 'intrusion_mean_pcl1', 'intrusion_std_pcl1',
            'intrusion_cutoff_pcl1', 'avoidance_pcl1', 'avoidance_mean_pcl1', 'avoidance_std_pcl1', 'avoidance_cutoff_pcl1', 
            'depression_pcl1', 'depression_mean_pcl1', 'depression_std_pcl1', 'depression_cutoff_pcl1', 'hypertention_pcl1',
            'hypertention_mean_pcl1', 'hypertention_std_pcl1', 'hypertention_cutoff_pcl1', 'tred_pcl1', 'tred_mean_pcl1', 
            'tred_std_pcl1', 'tred_cutoff_pcl1', 'only_avoidance_pcl1','only_avoidance_mean_pcl1', 'only_avoidance_std_pcl1', 
            'only_avoidance_cutoff_pcl1',
            
            'PCL_score_pcl2', 'PCL_mean_pcl2', 'PCL_std_pcl2', 'intrusion_pcl2', 'intrusion_mean_pcl2', 'intrusion_std_pcl2',
            'intrusion_cutoff_pcl2', 'avoidance_pcl2', 'avoidance_mean_pcl2', 'avoidance_std_pcl2', 'avoidance_cutoff_pcl2', 
            'depression_pcl2', 'depression_mean_pcl2', 'depression_std_pcl2', 'depression_cutoff_pcl2', 'hypertention_pcl2',
            'hypertention_mean_pcl2', 'hypertention_std_pcl2', 'hypertention_cutoff_pcl2', 'tred_pcl2', 'tred_mean_pcl2', 
            'tred_std_pcl2', 'tred_cutoff_pcl2', 'only_avoidance_pcl2','only_avoidance_mean_pcl2', 'only_avoidance_std_pcl2', 
            'only_avoidance_cutoff_pcl2'
]

In [6]:
discrete_features = ["age", "highschool_diploma", "dyslexia", "ADHD", "phq1", "lot1",
                    "trait1", "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2",
                    "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1",
                    "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2",
                    "planning2", "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                    "instrumental_support2", "self_distraction2", "denial2", "venting2", "substance_use2",
                    "behavioral_disengagement2", "self_blame2", "trauma_history8_1",
                    'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                    'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2',
                    'intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50']

In [7]:
genome_features = ["HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5"]
continuous_features = ["T1Acc1t", "T1Acc1n", "T1bias"]
t2_features = [
    "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2",
    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2", 
    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2', 'q6.5_PHYS_pcl2',
    'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2'
]
t1_features =[ "active_coping1","T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1",
                "trait1","state1",
                    "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
              "PCL1", "PCL_Broad1", "PCL_Strict1","cd_risc1", 
                'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1','q6.13_SLEEP_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1']

## fill the missing values

In [8]:
mtl_targets = ['intrusion_cutoff','hypertention_cutoff', 'avoidance_cutoff', 'depression_cutoff']
target_feature = ['PCL_Strict3']

In [9]:
df1 = df[features + mtl_targets + target_feature]

#df1 = df1.dropna(thresh=47)
df_t1 = df1[t1_features]
df_t2 = df1[t2_features]
df1 = df1[(df_t1.isna().astype(int).sum(axis=1)<38) | (df_t2.isna().astype(int).sum(axis=1)<38)]
df1["t1_missing"] = df_t1.isna().astype(int).sum(axis=1)==38
df1["t2_missing"] = df_t2.isna().astype(int).sum(axis=1)==38
features.extend(["t1_missing", "t2_missing"])

impute = 1
if impute:
    for i in mtl_targets:
        df1[i] = df[i].fillna(0)
    for i in mtl_targets + target_feature:
        print(i , df1[i].unique())
    mice = IterativeImputer()
    df1 = pd.DataFrame(mice.fit_transform(df1), columns=df1.columns)
    for i in mtl_targets + target_feature:
        print(i , df1[i].unique())
    #Y = Y.fillna(Y.mean())
    features.extend(["outliers_count_t1_35", "outliers_count_t2_35", "outliers_count_t1_25", "outliers_count_t2_25"])
    df1["outliers_count_t1_35"] = (np.abs(stats.zscore(df1[t1_features])) > 3.5).sum(axis=1)
    df1["outliers_count_t1_25"] = (np.abs(stats.zscore(df1[t1_features])) > 2.5).sum(axis=1)
    df1["outliers_count_t2_35"] = (np.abs(stats.zscore(df1[t2_features])) > 3.5).sum(axis=1)
    df1["outliers_count_t2_25"] = (np.abs(stats.zscore(df1[t2_features])) > 2.5).sum(axis=1)
    
    
#df1 = df1[(np.abs(stats.zscore(df1)) > 3).sum(axis=1)<15]

intrusion_cutoff [0. 1.]
hypertention_cutoff [0. 1.]
avoidance_cutoff [0. 1.]
depression_cutoff [0. 1.]
PCL_Strict3 [0. 1.]
intrusion_cutoff [0. 1.]
hypertention_cutoff [0. 1.]
avoidance_cutoff [0. 1.]
depression_cutoff [0. 1.]
PCL_Strict3 [0. 1.]


In [10]:
#df1[features] =StandardScaler().fit_transform(df1[features]), columns=df1.columns)
df1['std_genome'] = df1[genome_features].std(axis=1)
df1['std_t2'] = df1[[ "trait2", "state2", "PCL2"]].std(axis=1)
df1['std_t1'] = df1[[ "trait1", "state1", "PCL1"]].std(axis=1)

df1['std_pos_coping_t1'] = df1[["active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                "religion1", "emotional_support1", "instrumental_support1", "self_distraction1","venting1"]].std(axis=1)

df1['std_neg_coping_t1'] = df1[["denial1", "substance_use1", "behavioral_disengagement1", "self_blame1"]].std(axis=1)

df1['std_pos_coping_t2'] = df1[["active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                "religion1", "emotional_support2", "instrumental_support2", "self_distraction2","venting2"]].std(axis=1)

df1['std_neg_coping_t2'] = df1[["denial2", "substance_use2", "behavioral_disengagement2", "self_blame2"]].std(axis=1)

features = features + ['std_genome', 'std_t1', 'std_t2', 'std_pos_coping_t1', 'std_neg_coping_t1', 'std_pos_coping_t2', 'std_neg_coping_t2']

## target imbalance

In [11]:
for i in mtl_targets + target_feature:
    print(i , df1[i].unique())

intrusion_cutoff [0. 1.]
hypertention_cutoff [0. 1.]
avoidance_cutoff [0. 1.]
depression_cutoff [0. 1.]
PCL_Strict3 [0. 1.]


## cut off the test set

In [12]:
X, X_out, Y, y_out = train_test_split(df1[features + mtl_targets], df1[target_feature[0]],
                                      test_size=0.25, random_state=271828, stratify=df1[target_feature[0]])


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y, random_state=271828)
ss, tt = StandardScaler(), StandardScaler()
X_train, X_test = pd.DataFrame(np.hstack([ss.fit_transform(X_train[features]), X_train[mtl_targets]]), columns=X_train.columns), pd.DataFrame(np.hstack([tt.fit_transform(X_test[features]), X_test[mtl_targets]]), columns=X_test.columns)
cv = StratifiedKFold(5, random_state=271828)  

## the Model

## CV model of roc auc 

In [14]:
for activation in ['relu']: # ''
    for epochs in [500]: #200
        for loss_weights in [0.5, 0.8, 1.5, 3]:#, 2]: # 
            for patience in [15]:#, 23]:
                dropout_rate = 0.9
                print("\nepochs=", epochs, "\nactivation=", activation, "\npatience=", patience,
                      "\ndropout_rate =", dropout_rate, "\nloss_weight =", loss_weights)
                
                scores_f = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]
                    X_test_cv = X_train[features].iloc[test]
                    
                    
                           
                    ## smote
                   # columns=X_train_cv.columns
                   # sm = SMOTE(random_state=27)
                   # X_train_cv, y_train_cv = sm.fit_sample(X_train_cv , y_train_cv.ravel())
                    #X_train_cv = pd.DataFrame(X_train_cv, columns=columns)
                    
                    
                    targets = [X_train_cv[i] for i in mtl_targets] + [y_train_cv]
                    
                    X_train_cv.drop(mtl_targets, axis=1, inplace=True)
                    
                    class_weight = [{0:1, 1:6}] * len(mtl_targets) + [{0:1, 1:25}]
                
                   
                    ## dim reduction
                    #kpca = KernelPCA(n_components=20).fit(X_train_cv)
                    #X_train_cv = kpca.transform(X_train_cv)
                    #X_test_cv = kpca.transform(X_test_cv)
                    
                    ## feature selection
                    
                    #sfm = RFE(RandomForestClassifier(n_estimators=100), n_features_to_select=100).fit(X_train_cv, y_train_cv)
                    #X_train_cv = sfm.transform(X_train_cv)
                    #X_test_cv = sfm.transform(X_test_cv)
                    
                    visible = Input(shape=(X_train_cv.shape[1],))
                    x = Dense(20, activation=activation)(visible)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)

#                     x = Dense(20, activation=activation)(x)
#                     x = BatchNormalization()(x)
#                     x = Dropout(dropout_rate)(x)
                    
                    outputs = [Dense(1, activation='sigmoid')(x)] * (len(mtl_targets) + 1)
                    
                    es = EarlyStopping(monitor='val_loss', patience=patience)

                    model = Model(inputs=visible, outputs=outputs)

                    model.compile(loss=["binary_crossentropy"] * (len(mtl_targets) + 1),
                              optimizer='adam', loss_weights = [loss_weights] * len(mtl_targets) + [1])
                 
                
                    # class_weight=class_weight,
                    model.fit(X_train_cv, targets, epochs=epochs, class_weight=class_weight,
                              validation_split= 0.1, callbacks=[es], verbose=0)
                    y_pred = model.predict(X_test_cv)[len(mtl_targets)] >= 0.5                    

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                 #   scores_p.append(precision_score(y_train.iloc[test], y_pred))
                  #  scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("\n\nscores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))


epochs= 500 
activation= relu 
patience= 15 
dropout_rate = 0.9 
loss_weight = 0.5


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 



scores_f= 0.21746335545499793
mean auc= 0.7183424470266576

epochs= 500 
activation= relu 
patience= 15 
dropout_rate = 0.9 
loss_weight = 0.8


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 



scores_f= 0.2423708411574943
mean auc= 0.773028024606972

epochs= 500 
activation= relu 
patience= 15 
dropout_rate = 0.9 
loss_weight = 1.5


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 



scores_f= 0.22066646948634525
mean auc= 0.7400580997949419

epochs= 500 
activation= relu 
patience= 15 
dropout_rate = 0.9 
loss_weight = 3


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 



scores_f= 0.2274291936577073
mean auc= 0.7544634313055365


In [ ]:
for activation in ['elu']: # ''
    for epochs in [500]: #200
        for loss_weights in [0.1, 2]: # 
            for patience in [10, 23, 15]:
                dropout_rate = 0.9
                print("\nepochs=", epochs, "\nactivation=", activation, "\npatience=", patience,
                      "\ndropout_rate =", dropout_rate, "\nloss_weight =", loss_weights)
                
                scores_f = []
                #scores_p = []
                #scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(152,))
                    x = Dense(20, activation=activation)(visible)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)


                    outputs = [Dense(1, activation='sigmoid')(x)] * (len(mtl_targets) + 1)
                    
                    es = EarlyStopping(monitor='val_loss', patience=patience)

                    model = Model(inputs=visible, outputs=outputs)

                    model.compile(loss=["binary_crossentropy"] * (len(mtl_targets) + 1),
                              optimizer='adam', loss_weights = [loss_weights] * len(mtl_targets) + [1])
                    
                    targets = [X_train_cv[i] for i in mtl_targets] + [y_train_cv]
                    
                    X_train_cv.drop(mtl_targets, axis=1, inplace=True)
                    
                    class_weight = [{0:1, 1:4}] * len(mtl_targets) + [{0:1, 1:19}]
                
                    
                    model.fit(X_train_cv, targets, epochs=epochs, class_weight=class_weight, validation_split= 0.1, callbacks=[es], verbose=0)
                    y_pred = model.predict(X_train[features].iloc[test])[len(mtl_targets)] >= 0.5                    

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                 #   scores_p.append(precision_score(y_train.iloc[test], y_pred))
                  #  scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("\n\nscores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))

In [ ]:
for activation in ['elu']: # 'tanh'
    for epochs in [1000]: #200
        for loss_weights in [0.75]: # 
            for dropout_rate in [0.75, 0.9]:

                print("\nepochs=", epochs, "\nactivation=", activation,
                      "\ndropout_rate =", dropout_rate, "\nloss_weight =", loss_weights)
                
                scores_f = []
                #scores_p = []
                #scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(91,))
                    x = Dense(100, activation=activation)(visible)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)

                    x = Dense(70, activation=activation)(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    
                    x = Dense(5, activation=activation)(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)

                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)
                    output_diagnosis = Dense(1, activation='sigmoid')(x)

                    es = EarlyStopping(monitor='val_loss', patience=50)

                    model = Model(inputs=visible, outputs=[output_intrusion,
                            output_avoid, output_hype, output_50, output_diagnosis])

                    model.compile(loss=["binary_crossentropy"] * 5,
                              optimizer='adam', loss_weights = [loss_weights] * 4 + [1])
                    
                    targets = [X_train_cv['intrusion_cutoff'], X_train_cv['avoidance_cutoff'],
                               X_train_cv['hypertention_cutoff'], X_train_cv['regression_cutoff_50'], y_train_cv]
                    
                    X_train_cv.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50'], axis=1, inplace=True)
                    
                    class_weight = [{0:1, 1:4}, {0:1, 1:6}, {0:1, 1:4}, {0:1, 1:20}, {0:1, 1:19}]
                
                    
                    model.fit(X_train_cv, targets, epochs=epochs, class_weight=class_weight, validation_split= 0.1, callbacks=[es], verbose=0)
                    y_pred = model.predict(X_train[features].iloc[test])[4] >= 0.5                    

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                 #   scores_p.append(precision_score(y_train.iloc[test], y_pred))
                  #  scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("\n\nscores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))

In [ ]:
for i in range(3):
    for patience in [60]:#, 50, 25, 10, 5]: #200
        for loss_weights in [0.75]: # 
            for dropout_rate in [0.75]:
                activation = 'elu'
                epochs = 1000
                print("\nepochs=", epochs, "\nactivation=", activation,
                      "\ndropout_rate =", dropout_rate, "\nloss_weight =", loss_weights,
                     "\npatience =", patience)
                
                scores_f = []
                #scores_p = []
                #scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(91,))
                    x = Dense(70, activation=activation)(visible)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)

                    x = Dense(40, activation=activation)(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    
                   # x = Dense(20, activation=activation)(x)
                   # x = BatchNormalization()(x)
                   # x = Dropout(dropout_rate)(x)
                    
                    x = Dense(10, activation=activation)(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
              

                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)
                    output_diagnosis = Dense(1, activation='sigmoid')(x)

                    es = EarlyStopping(monitor='val_loss', patience=patience)

                    model = Model(inputs=visible, outputs=[output_intrusion,
                            output_avoid, output_hype, output_50, output_diagnosis])

                    model.compile(loss=["binary_crossentropy"] * 5,
                              optimizer='adam', loss_weights = [2* loss_weights,
                                                                loss_weights, loss_weights,1, 4*loss_weights])
                    
                    targets = [X_train_cv['intrusion_cutoff'], X_train_cv['avoidance_cutoff'],
                               X_train_cv['hypertention_cutoff'], X_train_cv['regression_cutoff_50'], y_train_cv]
                    
                    X_train_cv.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50'], axis=1, inplace=True)
                    
                    class_weight = [{0:1, 1:4}, {0:1, 1:6}, {0:1, 1:4}, {0:1, 1:20}, {0:1, 1:19}]
                
                    
                    model.fit(X_train_cv, targets, epochs=epochs, class_weight=class_weight, validation_split= 0.1, callbacks=[es], verbose=0)
                    y_pred = model.predict(X_train[features].iloc[test])[4] >= 0.5                    

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                 #   scores_p.append(precision_score(y_train.iloc[test], y_pred))
                  #  scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("\n\nscores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))